In [ ]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

In [ ]:
# notebook env value
subscription_id = '<SUBSCRIPTION_ID>'
acr_server = "<ACR_NAME>.azurecr.io"
acr_login = "<ACR_NAME>"
acr_pwd = "<ACR_PWD>"
acr_repo_name = "dw-tf-od:v1-gpu"

In [ ]:
# create workspace
from azureml.core import Workspace
ws = Workspace.create(name='dw-ws',
                      subscription_id=subscription_id,
                      resource_group='rg-aml-tf-od',
                      create_resource_group=True,
                      location='westus2' 
                     )

# Create the configuration file.
ws.write_config()

In [ ]:
# setup workspace - reloading
ws = Workspace.from_config()
ws.get_details()

In [ ]:
# setup datastore
ds = ws.get_default_datastore()
ds.name

In [ ]:
# upload data folder to ds
ds.upload(
    src_dir='./upload_data',
    target_path='tfdata',
    overwrite=True)

In [ ]:
# train remote VM - gpu
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-gpu')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-gpu', compute_config)
    compute_target.wait_for_completion(show_output=True)

In [ ]:
# train remote VM - cpu
"""from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-cpu')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-cpu', compute_config)
    compute_target.wait_for_completion(show_output=True)
"""

In [ ]:
# data reference
from azureml.core import Datastore
from azureml.core.runconfig import DataReferenceConfiguration

ds = Datastore.get(ws, datastore_name="workspaceblobstore")

dr_conf = DataReferenceConfiguration(
    datastore_name=ds.name,
    path_on_datastore='tfdata',
    #path_on_compute = '/tfdata'
    mode='mount') # or 'download'

In [ ]:
from azureml.core import Experiment
from azureml.core import Run
from azureml.core import ScriptRunConfig

In [ ]:
# setup azure container registry
from azureml.core.runconfig import ContainerRegistry

base_docker_image = ContainerRegistry()
base_docker_image.address = acr_server
base_docker_image.username = acr_login
base_docker_image.password = acr_pwd
tf_od_docker_image = acr_server + '/' + acr_repo_name

In [ ]:
# generate run config
from azureml.core.runconfig import RunConfiguration, DEFAULT_GPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment.docker.gpu_support = True #cpu, set False
run_config.environment.docker.base_image = tf_od_docker_image
run_config.environment.docker.base_image_registry = base_docker_image
run_config.data_references = {ds.name: dr_conf}
run_config.environment.python.user_managed_dependencies = True

run_config.target = compute_target.name
compute_target.name

In [ ]:
# run experiment
src = ScriptRunConfig(
    source_directory='./script',
    script='launcher.py',
    run_config=run_config,
    arguments=['--data_folder', str(ds.as_mount())] # only one param - data store mount dir
)
exp = Experiment(workspace=ws, name='dw-tf_remote_experiment_0417')
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)


In [ ]:
# status check
run.get_status()

In [ ]:
# cancel experiment
# run.cancel()

In [ ]:
# delete cpu compute
"""
mycompute = AmlCompute(workspace=ws, name='dw-cpu')
mycompute.delete()
"""

In [ ]:
# delete gpu compute
mycompute = AmlCompute(workspace=ws, name='dw-gpu')
mycompute.delete()

In [14]:
# delete workspace
ws.delete(delete_dependent_resources=True)